In [1]:
import time
import warnings

import numpy as np

warnings.filterwarnings('ignore') # Hide warnings that can generate and clutter notebook


def timing_experiment(x, y, distance_callable, distance_params = None, average=30):
    """Time the average time it takes to take the distance from the first time series
    to all of the other time series in X.

    Parameters
    ----------
    X: np.ndarray
        A dataset of time series.
    distance_callable: Callable
        A callable that is the distance function to time.

    Returns
    -------
    float
        Average time it took to run a distance
    """
    if distance_params is None:
        distance_params = {}
    total_time = 0
    for i in range(0, average):
        start = time.time()
        curr_dist = distance_callable(x, y, **distance_params)
        total_time += time.time() - start

    return total_time / average


In [ ]:
from sktime.distances.tests._utils import create_test_distance_numpy

from sktime.distances import distance_factory
from tslearn.metrics import dtw as tslearn_dtw
from dtw import dtw as dtw_python_dtw
from rust_dtw import dtw as rust_dtw_dtw
import pandas as pd

sktime_timing = []
tslearn_timing = []
dtw_python_timing = []
rust_dtw_timing = []
numba_sktime_timing = []

col_headers = []

univariate_timings_df = None

def univariate_experiment():
    increment_timepoints = 1000
    starting_num_timepoints = 1000
    ending_num_timepoints = 10000

    for i in range(starting_num_timepoints, ending_num_timepoints + increment_timepoints, increment_timepoints):
        col_headers.append(i)
        distance_m_d = create_test_distance_numpy(2, 1, i)

        x = distance_m_d[0][0]
        y = distance_m_d[1][0]
        numba_sktime = distance_factory(x, y, metric='dtw')

        tslearn_time = timing_experiment(x, y, tslearn_dtw)
        dtw_python_time = timing_experiment(x, y, dtw_python_dtw)
        rust_dtw_time = timing_experiment(x, y, rust_dtw_dtw, {'window':i, 'distance_mode': 'euclidean'})
        sktime_time = timing_experiment(distance_m_d[0], distance_m_d[1], numba_sktime)

        sktime_timing.append(sktime_time)
        tslearn_timing.append(tslearn_time)
        dtw_python_timing.append(dtw_python_time)
        rust_dtw_timing.append(rust_dtw_time)


    uni_df = pd.DataFrame({
        'time points': col_headers,
        'sktime': sktime_timing,
        'tslearn': tslearn_timing,
        'rust-dtw': rust_dtw_timing,
        'dtw-python': dtw_python_timing
    })
    return uni_df

univariate_timings_df = univariate_experiment()

In [ ]:
from sktime.distances.tests._utils import create_test_distance_numpy

from sktime.distances import distance_factory
from tslearn.metrics import dtw as tslearn_dtw
import pandas as pd

multivariate_timings_df = None

sktime_timing = []
tslearn_timing = []

col_headers = []

def multivariate_experiment():
    increment_timepoints = 100
    starting_num_timepoints = 100
    ending_num_timepoints = 500

    for i in range(starting_num_timepoints, ending_num_timepoints + increment_timepoints, increment_timepoints):
        col_headers.append(i)
        distance_m_d = create_test_distance_numpy(2, i, i)

        x = distance_m_d[0]
        y = distance_m_d[1]
        tslearn_x = x.reshape((x.shape[1], x.shape[0])) # tslearn wants m, d format
        tslearn_y = y.reshape((y.shape[1], y.shape[0])) # tslearn wants m, d format
        numba_sktime = distance_factory(x, y, metric='dtw')

        tslearn_time = timing_experiment(tslearn_x, tslearn_y, tslearn_dtw)
        sktime_time = timing_experiment(x, y, numba_sktime)

        sktime_timing.append(sktime_time)
        tslearn_timing.append(tslearn_time)


    multi_df = pd.DataFrame({
        'time points': col_headers,
        'sktime': sktime_timing,
        'tslearn': tslearn_timing,
    })
    return multi_df

multivariate_timings_df = multivariate_experiment()

In [ ]:
print(univariate_timings_df)

In [ ]:
print(multivariate_timings_df)

In [20]:
from sktime.distances.tests._utils import create_test_distance_numpy

from sktime.distances import dtw_distance
from tslearn.metrics import dtw as tslearn_dtw
from dtw import dtw as dtw_python_dtw
from rust_dtw import dtw as rust_dtw_dtw
import pandas as pd
import numpy as np

uni_size = 20
x_uni = create_test_distance_numpy(uni_size)
y_uni = create_test_distance_numpy(uni_size, random_state=2)

sktime_result = np.sqrt(dtw_distance(x_uni, y_uni)) # It is sqrt as isn't done in the distance
tslearn_result = tslearn_dtw(x_uni, y_uni)
rust_dtw_result = rust_dtw_dtw(x_uni, y_uni, window=uni_size, distance_mode='euclidean')
dtw_python_result = dtw_python_dtw(x_uni, y_uni)

print('sktime result: ', sktime_result)
print('tslearn result', tslearn_result)
print('rust-dtw result', rust_dtw_result)
print('dtw-python result', dtw_python_result.distance)

sktime result:  1.9081086144057442
tslearn result 1.9081086144057442
rust-dtw result 1.9081086144057442
dtw-python result 10.222175276151889
